# RoadSight — Dataset Exploration

This notebook is used to:
- Understand the RDD2020 dataset
- Inspect images and labels
- Learn how ML data is structured

No training happens here.

In [1]:
# We import standard Python libraries
# These help us interact with the operating system

import os
import sys

# Print Python version to ensure reproducibility
print("Python version:", sys.version)

# Print current working directory
# This helps us confirm where the notebook is running
print("Current directory:", os.getcwd())


Python version: 3.14.0 (tags/v3.14.0:ebf955d, Oct  7 2025, 10:15:03) [MSC v.1944 64 bit (AMD64)]
Current directory: c:\Users\saima\OneDrive\Desktop\RoadSight\RoadSight\ml\notebooks


In [1]:
import os

# Path to one country's images
image_dir = "../data/raw/RDD2020/Japan/images"

# List first 5 image files
images = os.listdir(image_dir)[:5]

print("Sample images:")
for img in images:
    print(img)


Sample images:
Japan_000000.jpg
Japan_000001.jpg
Japan_000002.jpg
Japan_000003.jpg
Japan_000004.jpg


In [4]:
import os
import xml.etree.ElementTree as ET

# Path to images and annotations
image_dir = "../data/raw/RDD2020/Japan/images"
label_dir = "../data/raw/RDD2020/Japan/annotations/xmls"

# Look at first 10 images (safe for testing)
image_files = sorted(os.listdir(image_dir))[:10]

for img in image_files:
    # Convert image filename to corresponding XML filename
    label_file = img.replace(".jpg", ".xml")
    label_path = os.path.join(label_dir, label_file)

    # Default: assume no damage
    label = 0

    # Check if annotation file exists
    if os.path.exists(label_path):
        # Parse the XML file
        tree = ET.parse(label_path)
        root = tree.getroot()

        # Find all <object> tags (each represents road damage)
        objects = root.findall("object")

        # If at least one object exists → damage present
        if len(objects) > 0:
            label = 1

    print(img, "→", label)


Japan_000000.jpg → 1
Japan_000001.jpg → 1
Japan_000002.jpg → 1
Japan_000003.jpg → 1
Japan_000004.jpg → 1
Japan_000005.jpg → 1
Japan_000006.jpg → 1
Japan_000008.jpg → 1
Japan_000009.jpg → 1
Japan_000010.jpg → 0


In [ ]:
import os

label_dir = "../data/raw/RDD2020/Japan/annotations/xmls"

xml_files = os.listdir(label_dir)[:10]

print("Sample annotation files:")
for f in xml_files:
    print(f)


Sample annotation files:
xmls


In [ ]:
import sys
import os
sys.path.append(os.path.abspath(".."))

from training.dataset import RoadSightDataset

image_dir = "../data/raw/RDD2020/Japan/images"
annotation_dir = "../data/raw/RDD2020/Japan/annotations/xmls"

# Initialize dataset
dataset = RoadSightDataset(image_dir, annotation_dir)

print("Total images:", len(dataset))

# Test first 5 examples
for i in range(5):
    img, label = dataset[i]
    print(f"Example {i} → shape: {img.shape}, label: {label}")


Total images: 10506
Example 0 → shape: torch.Size([3, 600, 600]), label: 1
Example 1 → shape: torch.Size([3, 600, 600]), label: 1
Example 2 → shape: torch.Size([3, 600, 600]), label: 1
Example 3 → shape: torch.Size([3, 600, 600]), label: 1
Example 4 → shape: torch.Size([3, 600, 600]), label: 1
Example 5 → shape: torch.Size([3, 600, 600]), label: 1
Example 6 → shape: torch.Size([3, 600, 600]), label: 1
Example 7 → shape: torch.Size([3, 600, 600]), label: 1
Example 8 → shape: torch.Size([3, 600, 600]), label: 1
Example 9 → shape: torch.Size([3, 600, 600]), label: 0
Example 10 → shape: torch.Size([3, 600, 600]), label: 1
Example 11 → shape: torch.Size([3, 600, 600]), label: 1
Example 12 → shape: torch.Size([3, 600, 600]), label: 1
Example 13 → shape: torch.Size([3, 600, 600]), label: 1
Example 14 → shape: torch.Size([3, 600, 600]), label: 1
Example 15 → shape: torch.Size([3, 600, 600]), label: 1
Example 16 → shape: torch.Size([3, 600, 600]), label: 1
Example 17 → shape: torch.Size([3, 108

In [4]:
import sys
import os
sys.path.append(os.path.abspath(".."))

import torch
from torch.utils.data import DataLoader
from torchvision import transforms
from training.dataset import RoadSightDataset


In [5]:
# Define image preprocessing steps
transform = transforms.Compose([
    transforms.Resize((224, 224)),           # Resize all images to 224x224
    transforms.ToTensor(),                    # Convert PIL image to tensor [C,H,W] in range [0,1]
    transforms.Normalize(                      # Normalize using ImageNet stats
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


In [6]:
image_dir = "../data/raw/RDD2020/Japan/images"
annotation_dir = "../data/raw/RDD2020/Japan/annotations/xmls"

dataset = RoadSightDataset(image_dir, annotation_dir, transform=transform)


In [7]:
# Batch size for training
batch_size = 32

# Create DataLoader
train_loader = DataLoader(
    dataset,
    batch_size=batch_size,
    shuffle=True,      # Shuffle for training
    num_workers=2      # Use 2 CPU threads for loading (adjust to your machine)
)


In [8]:
# Fetch one batch
images, labels = next(iter(train_loader))

print("Batch image tensor shape:", images.shape)
print("Batch labels:", labels)


Batch image tensor shape: torch.Size([32, 3, 224, 224])
Batch labels: tensor([1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        0, 1, 1, 1, 1, 1, 1, 1])
